In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
from paraguayfloodspy.pars import GetPars

## Data from IRI DL Angel

In [2]:
forecasts = ['Raw', 'XLR', 'PCR', 'CCA']
models = forecasts.copy(); models.remove('Raw')

In [3]:
obs_url = 'http://iridl.ldeo.columbia.edu/home/.agmunoz/.Paraguay/.Obs/.Obsrain_90thpctl/dods'
obs_exceed = xr.open_dataarray(obs_url)
obs_exceed = obs_exceed.isel(T=(obs_exceed['T'].size - 1))
hatch0 = np.ma.masked_invalid(obs_exceed.values)
hatch = np.ma.masked_less(hatch0, 0.9)
Xh, Yh = np.meshgrid(obs_exceed.X, obs_exceed.Y)

In [4]:
for i,m in enumerate(forecasts):
    url = 'http://iridl.ldeo.columbia.edu/ds%3A/home/.agmunoz/.Paraguay/.{}/.Odds_1-7Dec2015/dods'.format(m)
    ds = xr.open_dataarray(url)
    if len(ds.shape) > 2:
        ds = ds.isel(time = 0)
    if i == 0:
        odds = xr.Dataset({'{}'.format(m): ds})
    else:
        odds = odds.merge(xr.Dataset({'{}'.format(m): ds}))
    url = 'http://iridl.ldeo.columbia.edu/home/.agmunoz/.Paraguay/.{}/.2AFCProbFcsts/dods'.format(m)
    ds = xr.open_dataarray(url)
    if len(ds.shape) > 2:
        ds = ds.isel(time = 0)
    if i == 0:
        skill = xr.Dataset({'{}'.format(m): ds})
    else:
        skill = skill.merge(xr.Dataset({'{}'.format(m): ds}))

In [5]:
odds.to_netcdf("../_data/s2s/Forecast_Dec1_7.nc")
skill.to_netcdf("../_data/s2s/Skill_Dec1_7.nc")

## Box Forecasts

In [6]:
pars = GetPars('all')
pars

{'clim_plot': {'latmax': 10, 'latmin': -55, 'lonmax': 360, 'lonmin': 240},
 'months': [11, 12, 1, 2],
 'rain': {'latmax': 10, 'latmin': -55, 'lonmax': 330, 'lonmin': 270},
 'rpy_rain': {'latmax': -22.75,
  'latmin': -26.75,
  'lonmax': 304.75,
  'lonmin': 301.25},
 'time': {'eyear': 2016, 'syear': 1979},
 'wt_rgn': {'latmax': -15, 'latmin': -30, 'lonmax': 315, 'lonmin': 295}}

In [7]:
def GetURL(x0, x1, y0, y1):
    url = 'http://iridl.ldeo.columbia.edu/home/.mbell/.ECMWF/.S2Stest/.S2S/.ECMF_ph2/.forecast/.perturbed/.sfc_precip/.tp/'
    url += 'X/{}/{}/RANGEEDGES/'.format(x0, x1)
    url += 'Y/{}/{}/RANGEEDGES/'.format(y0, y1)
    url += '[X+Y]average/'
    url += 'L+differences/'
    url += 'dods'
    return(url)

In [8]:
url = GetURL(x0=pars['rpy_rain']['lonmin'],
             x1=pars['rpy_rain']['lonmax'],
             y0=pars['rpy_rain']['latmin'],
             y1=pars['rpy_rain']['latmax'])
print(url)

http://iridl.ldeo.columbia.edu/home/.mbell/.ECMWF/.S2Stest/.S2S/.ECMF_ph2/.forecast/.perturbed/.sfc_precip/.tp/X/301.25/304.75/RANGEEDGES/Y/-26.75/-22.75/RANGEEDGES/[X+Y]average/L+differences/dods


In [9]:
ds = xr.open_dataarray(url)
ds = ds.sel(S = slice('2015-09-01', '2016-02-29'))
ds['L'] = ds['L']
ds.to_netcdf("../_data/s2s/AreaAvg.nc")